In [98]:
import requests
import re
import csv
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import xml.etree.ElementTree as et
import numpy as np

In [29]:
#global styling display
pd.set_option('display.max_rows',None)
#pd.options.display.float_format = '{:,.0f}'.format

In [3]:
sec_url = 'https://www.sec.gov'
requested_cik = '0001067983'

In [4]:
def get_request(url):
    return requests.get(url)

def create_url(cik):
    return 'https://www.sec.gov/cgi-bin/browse-edgar?CIK={}&owner=exclude&action=getcompany&type=13F-HR'.format(cik)


In [5]:
# Find mutual fund by CIK number on EDGAR
response = get_request(create_url(requested_cik))
soup = BeautifulSoup(response.text, "html.parser")
tags = soup.findAll('a', id="documentsbutton")
Num_Files = len(tags)

In [90]:
def table13F(Num_File):
    # Find latest 13F report for mutual fund
    response_two = get_request(sec_url + tags[Num_File]['href'])
    soup_two = BeautifulSoup(response_two.text, "html.parser")
    tags_two = soup_two.findAll('a', attrs={'href': re.compile('xml')})
    xml_url = tags_two[3].get('href')
    response_xml = get_request(sec_url + xml_url)
    soup_xml = BeautifulSoup(response_xml.content, "lxml")

    #Extracting the filing and reporting date from the FormContent
    #print(soup_two.prettify())
    table_tag = soup_two.find('div', class_=['formContent'])
    table_tag2 = table_tag.find_all('div', class_=['infoHead','info'])

    FilingDate = table_tag2[1].text
    PeriodofReport = table_tag2[7].text

    rows = soup_xml.find_all('infotable')
    positions = []
    for row in rows:
        dic = {}
        position = row.find_all()
        dic["NAME_OF_ISSUER"] = position[0].text
        #dic["CUSIP"] = position[2].text
        #dic["$ VALUE"] = float(position[3].text.replace(',', ''))/1000
        dic["SHARES"] = float(position[5].text.replace(',', ''))/1000
        dic["PeriodofReport"]= table_tag2[7].text
        #dic["FilingDate"]= table_tag2[1].text
        positions.append(dic)
    df = pd.DataFrame(positions)
    return df

In [115]:
appended_table = []
for x in range(9):
    data = table13F(x)
    # store DataFrame in list
    appended_table.append(data)
# see pd.concat documentation for more info
appended_table = pd.concat(appended_table)
appended_table["PeriodofReport"] = pd.to_datetime(appended_table["PeriodofReport"])
#appended_table["FilingDate"] = pd.to_datetime(appended_table["FilingDate"])
appended_table = appended_table.groupby(["NAME_OF_ISSUER","PeriodofReport"]).sum(["SHARES"])

appended_table =pd.pivot_table(appended_table,index=["NAME_OF_ISSUER"],values=["SHARES"],
               columns=["PeriodofReport"],aggfunc=[np.sum],fill_value='-')

#styling --sequence is imp, has to come at last
# format_dict = {'SHARES':'{0:,.0f}k', 'PeriodofReport': '{:%m-%Y}'}
# my_styler = appended_table.style.format(format_dict)
# print(my_styler)
#appended_table.style.format({"SHARES":"{:,.0f}k" })
#appended_table=appended_table.unstack(level='PeriodofReport')
appended_table

sum                                   \
                                 SHARES                                    
PeriodofReport               2018-09-30 2018-12-31 2019-03-31 2019-06-30   
NAME_OF_ISSUER                                                             
ABBVIE INC                            -          -          -          -   
AMAZON COM INC                        -          -      483.3      537.3   
AMERICAN AIRLS GROUP INC          43700      43700      43700      43700   
AMERICAN EXPRESS CO              151611     151611     151611     151611   
APPLE INC                        252479     249589     249589     249589   
AXALTA COATING SYS LTD            24264      24264      24264      24264   
BANK AMER CORP                   877249     896168     896168     927249   
BANK OF NEW YORK MELLON CORP    77849.5    80937.3    80937.3    80937.3   
BARRICK GOLD CORPORATION              -          -          -          -   
BIOGEN INC                            -          -          -          -   
BRISTOL-MYERS SQUIBB CO               -          -          -          -   
CHARTER COMMUNICATIONS INC N    7340.99     7033.5    5710.71    5426.61   
COCA COLA CO                     400000     400000     400000     400000   
COSTCO WHSL CORP NEW            4333.36    4333.36    4333.36    4333.36   
DAVITA HEALTHCARE PARTNERS I    38565.6    38565.6    38565.6    38565.6   
DELTA AIR LINES INC DEL           65535      65535    70910.5    70910.5   
GENERAL MTRS CO                 52461.4    72269.7    72269.7    72269.7   
GLOBE LIFE INC                        -          -          -          -   
GOLDMAN SACHS GROUP INC         18353.6    18353.6    18353.6    18353.6   
JOHNSON & JOHNSON                 327.1      327.1      327.1      327.1   
JPMORGAN CHASE & CO             35664.8    50116.4    59514.9    59514.9   
KRAFT HEINZ CO                   325635     325635     325635     325635   
KROGER CO                             -          -          -          -   
LIBERTY GLOBAL PLC                27138      27138      27138      27138   
LIBERTY LATIN AMERICA LTD       3998.87    3998.87    3998.87    3998.87   
LIBERTY MEDIA CORP DELAWARE     45951.3    45951.3    45951.3    45951.3   
M & T BK CORP                   5382.04    5382.04    5382.04    5382.04   
MASTERCARD INC                  4934.76    4934.76    4934.76    4934.76   
MERCK & CO. INC                       -          -          -          -   
MONDELEZ INTL INC                   578        578        578        578   
MOODYS CORP                     24669.8    24669.8    24669.8    24669.8   
OCCIDENTAL PETE CORP                  -          -          -          -   
ORACLE CORP                     41404.8          -          -          -   
PFIZER INC                            -          -          -          -   
PHILLIPS 66                       15433    11895.8    5552.72    5552.72   
PNC FINL SVCS GROUP INC         6087.32    8263.06    8671.05    8671.05   
PROCTER & GAMBLE CO               315.4      315.4      315.4      315.4   
RED HAT INC                           -    4175.79    5110.47    5171.89   
RESTAURANT BRANDS INTL INC      8438.23    8438.23    8438.23    8438.23   
RH                                    -          -          -          -   
SIRIUS XM HLDGS INC              137916     137916     137916     137916   
SNOWFLAKE INC                         -          -          -          -   
SOUTHWEST AIRLS CO              56047.4    54847.4    53649.2    53649.2   
SPDR S&P 500 ETF TR                   -          -          -          -   
STONECO LTD                           -    14166.7    14166.7    14166.7   
STORE CAP CORP                  18621.7    18621.7    18621.7    18621.7   
SUNCOR ENERGY INC NEW                 -      10758      10758      10758   
SYNCHRONY FINL                    20803      20803      20803      20803   
T-MOBILE US INC                       -          -          -          -   
TEVA PHA